NOTE: To improve responses, especially for few-shot, the models that should be used are 7b and above.

## Credential HUGGINFACE

In [1]:
!pip install -U "transformers>=4.44.0" "huggingface_hub>=0.25.2"

  Using cached huggingface_hub-1.1.7-py3-none-any.whl.metadata (13 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#import os
#from google.colab import userdata
#userdata.get('HF_TOKEN')

from huggingface_hub import login

# Paste your copied token here
token = token_hugginface

login(token=token)

C:\Users\JOJO_\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Library

In [3]:
!pip install torch
!pip install numpy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import transformers
import pandas as pd
import numpy as np

# Input file

In [6]:
df = pd.read_csv('prototipo.csv')

# Prompts

## Sub Themes

In [7]:
fewshot = """You are an expert in classifying texts on diversity, equity, and inclusion using the following labels:

- gender: Statements related to women, men, non-binary people, gender equality, gender roles, or gender representation.
- race: Statements about ethnicity, racial minorities, discrimination based on race, or racial representation.
- LGBTQ+: Statements referring to sexual orientation, gender identity, same-sex relationships, or LGBTQ+ rights.
- inclusion: Statements about accessibility, disability inclusion, belonging, general DEIA practices, or inclusion that does not fit gender/race/LGBTQ+ specifically.
- none: Statements unrelated to DEIA or lacking any clear DEIA context.

Examples:
Input: "We need more women in leadership positions."
Output: gender

Input: "This policy supports minorities in the hiring process."
Output: race

Input: "The company is offering benefits for same-sex partners."
Output: LGBTQ+

Input: "We want a more inclusive workplace for everyone."
Output: inclusion

Input: "All employees must wear their ID badges."
Output: none

Classify the input using ONLY one of the following options: gender, race, LGBTQ+, inclusion, none.

Input: "{post}"
Output:"""

oneshot = """You are an expert in classifying texts on diversity, equity, and inclusion using the following labels:

    - gender: Statements related to women, men, non-binary people, gender equality, gender roles, or gender representation.
    - race: Statements about ethnicity, racial minorities, discrimination based on race, or racial representation.
    - LGBTQ+: Statements referring to sexual orientation, gender identity, same-sex relationships, or LGBTQ+ rights.
    - inclusion: Statements about accessibility, disability inclusion, belonging, general DEIA practices, or inclusion that does not fit gender/race/LGBTQ+ specifically.
    - none: Statements unrelated to DEIA or lacking any clear DEIA context.
    
    Example:

    Input: "We need more women in leadership positions."
    Output: gender

    Classify the input using only one the following options: gender, race, LGBTQ+, inclusion or none:

    Input: "{post}"
    Output:"""

zeroshot = """You are an expert in classifying texts on diversity, equity, and inclusion using the following labels:

- gender: statements related to women, men, non-binary people, gender equality, gender roles, or gender representation.
- race: statements about ethnicity, racial minorities, discrimination based on race, or racial representation.
- lgbtq+: statements referring to sexual orientation, gender identity, same-sex relationships, or lgbtq+ rights.
- inclusion: statements about accessibility, disability inclusion, belonging, general DEIA practices, or inclusion that does not fit gender/race/lgbtq+ specifically.
- none: statements unrelated to DEIA or lacking any clear DEIA context.

Classify the input using ONLY ONE of the following labels:
gender, race, lgbtq+, inclusion, none.

Respond with ONLY the label, in lowercase, with no explanation.

Input: "{post}"
Output:"""


## Pro Anti

In [8]:
fewshot_polarization="""You are an expert in classifying texts on diversity, equity, and inclusion using the following labels:

- pro-DEIA: Supports, encourages, or speaks favorably about DEIA principles or actions.
- anti-DEIA: Opposes, criticizes, mocks, minimizes, or rejects DEIA principles.
- neutral: Unrelated, ambiguous, factual, or lacking any clear stance toward DEIA.
Examples:

Input: "We need more women and people of color in leadership roles."
Output: pro-DEIA

Input: "Hiring should be based on merit, not race or gender."
Output: anti-DEIA

Input: "Our company is hosting a webinar on leadership strategies."
Output: neutral

Input: "Quotas are a form of reverse discrimination."
Output: anti-DEIA

Input: "I don't see color, I treat everyone the same."
Output: anti-DEIA

Input: "Accessibility tools like screen readers help all users, not just those with disabilities."
Output: pro-DEIA

Input: "We should have more inclusive hiring practices."
Output: pro-DEIA


Input: "Hiring should be based on merit, not quotas."
Output: anti-DEIA

Classify the input using ONLY one of the following labels: pro-DEIA, anti-DEIA, neutral.

Input: "{post}"
Output:"""


oneshot_polarization="""You are an expert in classifying texts on diversity, equity, and inclusion using the following labels:

- pro-DEIA: Supports, encourages, or speaks favorably about DEIA principles or actions.
- anti-DEIA: Opposes, criticizes, mocks, minimizes, or rejects DEIA principles.
- neutral: Unrelated, ambiguous, factual, or lacking any clear stance toward DEIA.
Example:

Input: "I don't see color, I treat everyone the same."
Output: anti-DEIA

Classify the input using ONLY one of the following labels: pro-DEIA, anti-DEIA, neutral.

Input: "{post}"
Output:"""

zeroshot_polarization="""You are an expert in classifying texts on diversity, equity, and inclusion using the following labels:

- pro-DEIA: Supports, encourages, or speaks favorably about DEIA principles or actions.
- anti-DEIA: Opposes, criticizes, mocks, minimizes, or rejects DEIA principles.
- neutral: Unrelated, ambiguous, factual, or lacking any clear stance toward DEIA.
Classify the input using ONLY one of the following labels: pro-DEIA, anti-DEIA, neutral.

Input: "{post}"
Output:"""



# Setup LLM

In [9]:
!pip install accelerate


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model_name= "mistral"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    dtype="auto",
    device_map="auto",
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 41.67it/s]
Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu


## Function to send the post and the prompt for LLM analysis

In [11]:
def analisys(post,prompt):
    output = pipe(prompt, max_new_tokens=40, do_sample=True, temperature=0.3)
    output=output[0]['generated_text']
    result=output.split("Output:")[-1].strip().lower()
    print(result)
    print("""""""")
    return result.split(",")[0]

# Classification

In [12]:
# Create a prompt list for bulk execution

prompt = [zeroshot,oneshot,fewshot]

# To avoid conflict, we called the second list message
message = [zeroshot_polarization,oneshot_polarization,fewshot_polarization]

In [13]:
# Execute all prompts: Sub themes and Pro anti.

for p in range(len(prompt)):
  execute = prompt[p]
  df['prompt_'+model_name+'_'+str(p)] = df['Text'].apply(lambda x: analisys(x, execute))
  execute = message[p]
  df['message_'+model_name+'_'+str(p)] = df['Text'].apply(lambda x: analisys(x, execute))



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{post}



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{label}



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

neutral



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


anti



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


neutral

{label}



# Save output to CSV file

In [14]:
df.to_csv("result_"+model_name+".csv", index = False)